In [26]:
import pandas as pd
import datetime as dt
import numpy as np
from sklearn.model_selection import train_test_split
import math

import warnings
warnings.filterwarnings('ignore')

In [27]:
def acquire_data():
    df = pd.read_csv('Traffic_Violations_montgomery_county.csv')
    return df

In [28]:
df = acquire_data()
df.head()

,SeqID,Date Of Stop,Time Of Stop,Agency,SubAgency,Description,Location,Latitude,Longitude,Accident,...,Charge,Article,Contributed To Accident,Race,Gender,Driver City,Driver State,DL State,Arrest Type,Geolocation
0,fbc324ab-bc8d-4743-ba23-7f9f370005e1,08/11/2019,20:02:00,MCP,"2nd District, Bethesda","LEAVING UNATTENDED VEH. W/O STOPPING ENGINE, L...",CORDELL ST @ NORFOLK AVE.,38.989743,-77.097770,No,...,21-1101(a),Transportation Article,False,BLACK,M,SILVER SPRING,MD,MD,A - Marked Patrol,"(38.9897433333333, -77.09777)"
1,a6d904ec-d666-4bc3-8984-f37a4b31854d,08/12/2019,13:41:00,MCP,"2nd District, Bethesda",EXCEEDING POSTED MAXIMUM SPEED LIMIT: 85 MPH I...,NBI270 AT MIDDLEBROOK RD,39.174110,-77.246170,No,...,21-801.1,Transportation Article,False,WHITE,M,SILVER SPRING,MD,MD,A - Marked Patrol,"(39.17411, -77.24617)"
2,54a64f6a-df28-4b65-a335-08883866aa46,08/12/2019,21:00:00,MCP,"5th District, Germantown",DRIVING VEH W/ TV-TYPE RECEIVING VIDEO EQUIP T...,MIDDLEBROOK AN 355,39.182015,-77.238221,No,...,21-1129,Transportation Article,False,BLACK,M,GAITHERSBURG,MD,MD,A - Marked Patrol,"(39.1820155, -77.2382213333333)"
3,cf5479b6-9bc7-4216-a7b2-99e57ae932af,08/12/2019,21:43:00,MCP,"5th District, Germantown",DRIVING VEHICLE ON HIGHWAY WITH SUSPENDED REGI...,GERMANTOWN RD AND ALE HOUSE,39.160508,-77.284023,No,...,13-401(h),Transportation Article,False,BLACK,M,GERMANTOWN,MD,MD,A - Marked Patrol,"(39.1605076666667, -77.284023)"
4,5601ca35-8ee7-4f8e-9208-d89cde96d469,08/12/2019,21:30:00,MCP,"2nd District, Bethesda",FAILURE OF LICENSEE TO NOTIFY ADMINISTRATION O...,EASTWEST/ 355,38.984247,-77.090548,No,...,16-116(a),Transportation Article,False,BLACK,M,SILVER SPRING,MD,MD,A - Marked Patrol,"(38.9842466666667, -77.0905483333333)"


## Clean Data

1. drop columns
2. check null value
3. convert time format
4. set the time to index and create new columns for year, month, day of the week and hour
5. convert the boolean value into int: Contributed to accident
6. drop rows: SubAgency S15 only has 3 rows and W15 only have 7 rows.
7. get all the data related to accident.

In [29]:
# drop columns
drop_col = ['SeqID', 'Agency', 'Accident', 'Fatal', 'Commercial License','HAZMAT', 'Commercial Vehicle', 
           'Work Zone', 'State', 'VehicleType', 'Year', 'Make', 'Model', 'Color', 'Charge', 'Article', 'Driver State', 'DL State', 'Search Reason For Stop', 'Search Arrest Reason', 'Location',
           'Search Conducted', 'Search Disposition', 'Search Outcome', 'Search Reason', 'Search Type','Geolocation',
           'Latitude', 'Longitude','Description', 'Driver City', 'Arrest Type']
df = df.drop(columns = drop_col)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1803695 entries, 0 to 1803694
Data columns (total 11 columns):
 #   Column                   Dtype 
---  ------                   ----- 
 0   Date Of Stop             object
 1   Time Of Stop             object
 2   SubAgency                object
 3   Belts                    object
 4   Personal Injury          object
 5   Property Damage          object
 6   Alcohol                  object
 7   Violation Type           object
 8   Contributed To Accident  bool  
 9   Race                     object
 10  Gender                   object
dtypes: bool(1), object(10)
memory usage: 139.3+ MB


In [7]:
# check null value
df.isnull().sum()

Date Of Stop               0
Time Of Stop               0
SubAgency                  0
Belts                      0
Personal Injury            0
Property Damage            0
Alcohol                    0
Violation Type             0
Contributed To Accident    0
Race                       0
Gender                     0
dtype: int64

In [30]:
# convert time format
df['date_time'] = df['Date Of Stop'] + ' ' + df['Time Of Stop']
col = ['Date Of Stop', 'Time Of Stop']
df = df.drop(columns = col)
df['date_time'] = pd.to_datetime(df['date_time'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1803695 entries, 0 to 1803694
Data columns (total 10 columns):
 #   Column                   Dtype         
---  ------                   -----         
 0   SubAgency                object        
 1   Belts                    object        
 2   Personal Injury          object        
 3   Property Damage          object        
 4   Alcohol                  object        
 5   Violation Type           object        
 6   Contributed To Accident  bool          
 7   Race                     object        
 8   Gender                   object        
 9   date_time                datetime64[ns]
dtypes: bool(1), datetime64[ns](1), object(8)
memory usage: 125.6+ MB


In [31]:
# set the time to index and create new columns for year, month, day of the week and hour
df.set_index('date_time', inplace=True)
df.sort_index(inplace=True)
df['month'] = df.index.strftime('%m-%b')
df['day_of_week'] = df.index.strftime('%A')
df['year'] = df.index.strftime('%Y')
df['hour'] = df.index.strftime('%H')
df.head()

,SubAgency,Belts,Personal Injury,Property Damage,Alcohol,Violation Type,Contributed To Accident,Race,Gender,month,day_of_week,year,hour
date_time,,,,,,,,,,,,,
2012-01-01 00:43:00,Headquarters and Special Operations,No,No,No,No,Warning,False,WHITE,M,01-Jan,Sunday,2012,00
2012-01-01 00:52:00,"3rd District, Silver Spring",No,No,No,No,Citation,False,HISPANIC,F,01-Jan,Sunday,2012,00
2012-01-01 00:52:00,"3rd District, Silver Spring",No,No,No,No,Citation,False,HISPANIC,F,01-Jan,Sunday,2012,00
2012-01-01 00:52:00,"3rd District, Silver Spring",No,No,No,No,Citation,False,HISPANIC,F,01-Jan,Sunday,2012,00
2012-01-01 00:52:00,"3rd District, Silver Spring",No,No,No,No,Citation,False,HISPANIC,F,01-Jan,Sunday,2012,00


In [10]:
df.SubAgency.value_counts()

4th District, Wheaton                              425851
3rd District, Silver Spring                        344000
2nd District, Bethesda                             288231
6th District, Gaithersburg / Montgomery Village    229228
5th District, Germantown                           211701
1st District, Rockville                            211135
Headquarters and Special Operations                 93539
W15                                                     7
S15                                                     3
Name: SubAgency, dtype: int64

In [32]:
# convert the boolean value into int
df['Contributed To Accident'] = np.where(df['Contributed To Accident'] == True, 1, df['Contributed To Accident'])
df['Contributed To Accident'] = np.where(df['Contributed To Accident'] == False, 0, df['Contributed To Accident'])

# SubAgency S15 and W15 only have less than 10 rows.
df = df[df.SubAgency != 'S15']
df = df[df.SubAgency != 'W15']

In [12]:
df = df[df['Contributed To Accident'] == 1]
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 782889 entries, 2017-01-01 00:00:00 to 2022-08-13 21:51:00
Data columns (total 13 columns):
 #   Column                   Non-Null Count   Dtype 
---  ------                   --------------   ----- 
 0   SubAgency                782889 non-null  object
 1   Belts                    782889 non-null  object
 2   Personal Injury          782889 non-null  object
 3   Property Damage          782889 non-null  object
 4   Alcohol                  782889 non-null  object
 5   Violation Type           782889 non-null  object
 6   Contributed To Accident  782889 non-null  int64 
 7   Race                     782889 non-null  object
 8   Gender                   782889 non-null  object
 9   month                    782889 non-null  object
 10  day_of_week              782889 non-null  object
 11  year                     782889 non-null  object
 12  hour                     782889 non-null  object
dtypes: int64(1), object(12)
memory usage: 83

### create a function for cleaning data

In [13]:
def clean_data(df):
    # drop columns
    drop_col = ['SeqID', 'Agency', 'Accident', 'Fatal', 'Commercial License','HAZMAT', 'Commercial Vehicle', 
                'Work Zone', 'State', 'VehicleType', 'Year', 'Make', 'Model', 'Color', 'Charge', 'Article', 
                'Driver State', 'DL State', 'Search Reason For Stop', 'Search Arrest Reason', 'Location',
                'Search Conducted', 'Search Disposition', 'Search Outcome', 'Search Reason', 'Search Type',
                'Geolocation','Latitude', 'Longitude','Description', 'Driver City', 'Arrest Type']
    df = df.drop(columns = drop_col)
    
    # # convert time format
    df['date_time'] = df['Date Of Stop'] + ' ' + df['Time Of Stop']
    col = ['Date Of Stop', 'Time Of Stop']
    df = df.drop(columns = col)
    df['date_time'] = pd.to_datetime(df['date_time'])
    
    # set the time to index and create new columns for year, month, day of the week and hour
    df.set_index('date_time', inplace=True)
    df.sort_index(inplace=True)
    df['month'] = df.index.strftime('%m-%b')
    df['day_of_week'] = df.index.strftime('%A')
    df['year'] = df.index.strftime('%Y')
    df['hour'] = df.index.strftime('%H')
    
    # convert the boolean value into int
    df['Contributed To Accident'] = np.where(df['Contributed To Accident'] == True, 1, df['Contributed To Accident'])
    df['Contributed To Accident'] = np.where(df['Contributed To Accident'] == False, 0, df['Contributed To Accident'])

    # SubAgency S15 and W15 only have less than 10 rows.
    df = df[df.SubAgency != 'S15']
    df = df[df.SubAgency != 'W15']
    
    # get all the data related to accident
    df = df[df['Contributed To Accident'] == 1]
    
    return df

## Split Data

In [ ]:
# split data
train_validate, test = train_test_split(df, test_size=.2, 
                                        random_state=123)

train, validate = train_test_split(train_validate, test_size=.3, 
                                   random_state=123)

In [14]:
# function of splitting data

def split_data(df):
    train_validate, test = train_test_split(df, test_size=.2, 
                                        random_state=123)

    train, validate = train_test_split(train_validate, test_size=.3, 
                                       random_state=123)
    
    return train, validata, test

In [15]:
# function pf preparing data

def prep_data(df):
    
    df = clean_data(df)
    train, validata, test = split_data(df)
    return train, validata, test

### Function for calculating F1 score

In [ ]:
from sklearn.metrics import confusion_matrix
def f1_score(y, prediction):
    TN, FP, FN, TP = confusion_matrix(y, prediction).ravel()
    ALL = TP + FP + FN + TN

    true_positive_rate = sensitivity = recal = power = TP/(TP+FN)
    precision = PPV = TP/(TP+FP)
    f1 = 2*(precision*recal)/(precision+recal)
    return f1